In [8]:
import os
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\Chris Partee\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [13]:
mars_data = {}

In [11]:
# # Enter whatever URL you like
browser.visit("https://redplanetscience.com/")

# Add the page source to the variable `content`.
content = browser.html
# Load the contents of the page, its source, into BeautifulSoup 

soup = BeautifulSoup(content,features="lxml")
# print (soup.prettify)

In [16]:
# Object is “results”, brackets make the object an empty list.
# We will be storing our data here.
results = []
for element in soup.findAll(attrs={'class': 'list_text'}):
    news={}
    title = element.find(attrs={'class':'content_title'})
#     print (title.text)
    para = element.find(attrs={'class':'article_teaser_body'})
#     print (para.text)
    news['news_title']=title.text
    news['news_p']=para.text

    results.append(news)

mars_data["news"] = results
print(results)

[{'news_title': "Air Deliveries Bring NASA's Perseverance Mars Rover Closer to Launch", 'news_p': "A NASA Wallops Flight Facility cargo plane transported more than two tons of equipment — including the rover's sample collection tubes — to Florida for this summer's liftoff."}, {'news_title': "NASA's Curiosity Mars Rover Snaps Its Highest-Resolution Panorama Yet", 'news_p': 'To go along with the stunning 1.8-billion-pixel image, a new video offers a sweeping view of the Red Planet.'}, {'news_title': "HiRISE Views NASA's InSight and Curiosity on Mars", 'news_p': 'New images taken from space offer the clearest orbital glimpse yet of InSight as well as a view of Curiosity rolling along.'}, {'news_title': "NASA's Curiosity Rover Finds an Ancient Oasis on Mars", 'news_p': "New evidence suggests salty, shallow ponds once dotted a Martian crater — a sign of the planet's drying climate."}, {'news_title': "Mars Helicopter Attached to NASA's Perseverance Rover", 'news_p': "The team also fueled the

In [22]:
browser.visit("https://spaceimages-mars.com/")

# Add the page source to the variable `content`.
content = browser.html
# Load the contents of the page, its source, into BeautifulSoup 
# class, which analyzes the HTML as a nested data structure and allows to select
# its elements by using various selectors.
soup = BeautifulSoup(content,features="lxml")
print(soup.prettify)

<bound method Tag.prettify of <html class=""><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
<style type="text/css">.fancybox-margin{margin-right:17px;}</style></head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-

In [23]:
imgtag = soup.find(attrs={'class':'headerimage fade-in'})
featured_image_url = imgtag['src']

In [24]:
mars_data["space_img"] = featured_image_url

In [31]:
#pulling table using pandas to scrap
#then exporting to be used later in flask
tables = pd.read_html("https://galaxyfacts-mars.com")
mars_facts = tables[1]
mars_facts.columns = ["Data Type", "Info"]
mars_facts.drop(index=mars_facts.index[0], axis=0,inplace=True)
mars_facts = mars_facts.set_index("Data Type")

In [32]:
mars_data["facts_table"] = mars_facts.to_html()

In [36]:
#connecting to the hemisphere page
browser.visit("https://marshemispheres.com/")
content = browser.html

soup = BeautifulSoup(content,features="lxml")

In [37]:
#grabbing the headers text of images on home page that have links
mars_hemispheres = []

for x in soup.findAll("div", attrs={"class": "item"}):
    ltext = x.find("h3")
    if ltext:
        mars_hemispheres.append(ltext.text)


In [38]:
#making connection with browser to run the click loop
browser.visit("https://marshemispheres.com/")

In [39]:
#click loop that finds the name and url of the original images
hemis_imgs = []

for m in mars_hemispheres:
    hemis = {}
    
    browser.find_by_text(m).first.click()
    visit = browser.links.find_by_partial_href(".tif").first
    hemis["tif_url"] = visit["href"]
    hemis["title"] = browser.find_by_css("h2.title").text
    
    hemis_imgs.append(hemis)
    
    browser.back()
    
    
browser.quit()

In [40]:
mars_data["hemi_pics"] = hemis_imgs